# Creating a Simple Agent with Tracing

In [16]:
import dotenv
import os

from openai import OpenAI
from notebooks.guardrails import nutrition_agent

dotenv.load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    print(
        """Error: OPENAI_API_KEY environment variable not set. Please copy the .env.template file as .env and fill it in.
    
    You can execute these commands in the terminal to get started:
    cp .env.template .env
    code .env
    """
    )

# Test OpenAI Access
print(
    OpenAI()
    .responses.create(
        model=os.environ["OPENAI_DEFAULT_MODEL"], input="Say: We are up and running!"
    )
    .output_text
)

We are up and running!


In [17]:
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent

Create a simple Nutrition Assistant Agent

In [18]:
nutrition_agent = Agent(name="Nutrition Assistant", instructions="Give me nutrition facts for a given food.")

Let's execute the Agent:

In [19]:
with trace("Simple Nutrition Agent"):
    result = await Runner.run(nutrition_agent, "How healthy are bananas?")

print(result)

RunResult:
- Last agent: Agent(name="Nutrition Assistant", ...)
- Final output (str):
    Here are the nutrition basics for bananas.
    
    Per medium banana (about 118 g):
    - Calories: ~105
    - Carbohydrates: ~27 g
      - Sugars: ~14 g
      - Fiber: ~3 g
    - Protein: ~1 g
    - Fat: ~0.4 g
    - Potassium: ~422 mg (about 12% DV)
    - Vitamin C: ~10 mg (about 12% DV)
    - Vitamin B6: ~0.5 mg (about 25% DV)
    - Magnesium: ~32 mg
    
    Per 100 g (roughly):
    - Calories: ~89
    - Carbohydrates: ~23 g
      - Sugars: ~12 g
      - Fiber: ~2.6 g
    - Protein: ~1.1 g
    - Fat: ~0.3 g
    - Potassium: ~358 mg
    - Vitamin C: ~8.7 mg
    - Vitamin B6: ~0.37 mg
    
    Health take:
    - Benefits: good source of potassium, vitamin C, vitamin B6, and dietary fiber; supports heart health and digestion.
    - Considerations: bananas have natural sugars; portion size matters for those watching sugar intake or blood sugar (ripeness can affect sugar and resistant starch conte

Streaming the answer to the screen, token by token

In [20]:
response_stream = Runner.run_streamed(nutrition_agent, "How healthy are bananas?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(
        event.data, ResponseTextDeltaEvent
    ):
        print(event.data.delta, end="", flush=True)

Here are typical nutrition facts for one medium banana (about 118 grams):

- Calories: ~105
- Carbohydrates: ~27 g
  - Sugars: ~14 g
  - Fiber: ~3 g
- Protein: ~1.3 g
- Fat: ~0.3 g
- Potassium: ~422 mg
- Vitamin C: ~10 mg
- Vitamin B6: ~0.5 mg
- Magnesium: ~32 mg

Health notes:
- Good source of potassium and vitamin B6; supports heart and nerve health.
- Contains dietary fiber and resistant starch (in greener bananas) that aid digestion.
- Provides natural sugars; ripeness affects sugar content (ripe bananas are sweeter).
- Glycemic index is moderate and can vary with ripeness and individual response.

Overall: bananas are a healthy, convenient fruit when eaten as part of a balanced diet, but portion size may matter for those monitoring sugar or total carbohydrate intake.

_Good Job!_